### Ejercicio de Webscraping

In [3]:
# Importar las librerias que usaremos

import pandas as pd #Nod ayudará a hacer un DF con la información obtenida
from selenium import webdriver #Permite automatizar navegadores web para interactuar con sitios web (como abrir páginas, hacer clic en botones o llenar formularios).
from bs4 import BeautifulSoup as bs #para analizar y manipular documentos HTML o XML de manera sencilla. Es ideal para extraer datos específicos de un sitio web.
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager # Se encarga de descargar automáticamente el controlador (driver) correcto para el navegador que estás usando (en este caso, Chrome). Evita que tengas que descargar manualmente el chromedriver compatible con tu navegador.
from urllib.parse import urljoin

# Configuración

# Configuramos el driver de chrome
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

titulos = []
precios = []
stocks = []

url = "http://books.toscrape.com"

while True:
    titulos_sucios = []
    precios_sucios = []
    stocks_sucio = []

    # urlaux = urljoin(url,f"/catalogue/page-{page}.html")

    driver.get(url)

    # Extracción de datos
    contenido = driver.page_source

    soup = bs(contenido)

    articles = soup.find_all("article",attrs={"class":"product_pod"})

    for libro in articles:
        titulo = libro.find("h3")
        titulos_sucios.append(titulo.text)

        precio = libro.find("p",attrs={"class":"price_color"})
        precios_sucios.append(precio.text)

        stock = libro.find("p",attrs={"class":"instock availability"})
        stocks_sucio.append(stock.text)

    # Limpiando los datos, Se eliminan caracteres no deseados (como saltos de línea).
    for libro in titulos_sucios:
        titulos.append(libro.replace("\n","").strip())

    for libro in precios_sucios:
        precios.append(libro.replace("\n","").strip())

    for libro in stocks_sucio:
        stocks.append(libro.replace("\n","").strip())
    
    siguientepag = soup.select_one("li.next>a")
    if siguientepag:
        siguienteurl = siguientepag.get("href")
        url = urljoin(url, siguienteurl)
    else:
        break


# Almacenamiento datos
#Los datos se almacenan en un DataFrame de pandas y luego se exportan a un archivo .csv.
df = pd.DataFrame({"titulo":titulos,"precio":precios,"stock":stocks})
df.head()

# Exportando los datos a un .csv
df.to_csv("libros.csv", index=False, encoding="utf-8")